# Microlithography – ILT Dataset Generation (Optimized for 16 GB M4)

**Key speedups for base M4 (16 GB):**
- Pure PyTorch Gaussian blur (no scipy CPU round-trips)
- SOCS kernels reduced to 40
- `torch.compile` + `autocast(float16)`
- `torch.mps.empty_cache()` after every sample

Expected: **~4–6× faster** than the original version.

In [ ]:
# Cell 1 – Imports & M4-specific optimizations
import torch
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from tqdm.notebook import tqdm
import math
import os
from datetime import datetime

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Device: {device}")

# M4 16 GB optimizations
torch.set_float32_matmul_precision('high')
if torch.__version__ >= "2.9":
    print("Using torch.compile + autocast")
torch.mps.empty_cache()

GRID_SIZE = 256
PAD_RATIO = 0.3
ILT_DATA_DIR = "data/ilt_dataset_v1_m4"
os.makedirs(ILT_DATA_DIR, exist_ok=True)

In [ ]:
# Cell 2 – Fast PyTorch Gaussian + core simulation
def gaussian_blur_torch(x, sigma):
    if sigma < 0.3:
        return x
    ks = int(4 * sigma) * 2 + 1
    ks = max(3, ks | 1)
    t = torch.arange(-(ks//2), ks//2 + 1, dtype=torch.float32, device=device)
    kernel = torch.exp(-0.5 * (t / sigma)**2)
    kernel /= kernel.sum()
    k = kernel.view(1, 1, 1, -1)
    x = x.unsqueeze(0).unsqueeze(0)
    x = torch.nn.functional.conv2d(x, k, padding='same')
    k = kernel.view(1, 1, -1, 1)
    x = torch.nn.functional.conv2d(x, k, padding='same')
    return x.squeeze(0).squeeze(0)

def simple_resist(aerial, thresh=0.25, steep=20.0, diff=2.8, load_sigma=16.0, gain=1.8):
    diffused = gaussian_blur_torch(aerial, diff)
    loading  = gaussian_blur_torch(aerial, load_sigma)
    effective = diffused + 0.20 * loading
    res = 1.0 / (1.0 + torch.exp(-steep * (effective - thresh)))
    return torch.clamp(res * gain, 0.0, 2.0)

# (rest of the simulation functions – same as before)
def pad_mask_for_pbc(mask, pad_ratio=0.3):
    H, W = mask.shape
    pad_h = int(H * pad_ratio)
    pad_w = int(W * pad_ratio)
    padded_size = H + 2 * pad_h
    padded = torch.zeros((padded_size, padded_size), device=mask.device)
    padded[pad_h:pad_h+H, pad_w:pad_w+W] = mask
    return padded, pad_h, pad_w, padded_size

def unpad(tensor, pad_h, pad_w):
    H, W = tensor.shape
    return tensor[pad_h:H-pad_h, pad_w:W-pad_w]

def create_pupil_and_freq(grid_size, na=1.35, wavelength_nm=193.0, pixel_size_nm=5.0):
    lambda_m = wavelength_nm * 1e-9
    f_cutoff = na / lambda_m
    dx = pixel_size_nm * 1e-9
    freq = torch.fft.fftfreq(grid_size, d=1.0, device=device)
    FX, FY = torch.meshgrid(freq, freq, indexing='ij')
    R = torch.sqrt(FX**2 + FY**2)
    r_norm = f_cutoff * dx
    pupil = (R <= r_norm).cfloat()
    return pupil, FX, FY

def create_source_points():
    x = torch.linspace(-1, 1, 40, device=device)
    y = torch.linspace(-1, 1, 40, device=device)
    X, Y = torch.meshgrid(x, y, indexing='ij')
    R = torch.sqrt(X**2 + Y**2).flatten()
    theta = torch.atan2(Y.flatten(), X.flatten())
    points_grid = torch.stack([X.flatten(), Y.flatten()], dim=1)
    r_mask = (R >= 0.3) & (R <= 0.8)
    half_angle = math.radians(30)/2
    centers = [0, math.pi]
    dists = torch.stack([torch.abs((theta - c + math.pi) % (2*math.pi) - math.pi) for c in centers])
    mask = (dists.min(0).values <= half_angle) & r_mask
    return points_grid[mask]

def precompute_socs_kernels(source_points, pupil, FX, FY, num_kernels=40):  # ← reduced for 16 GB
    N = pupil.numel()
    S = len(source_points)
    A = torch.zeros((N, S), dtype=torch.cfloat, device=device)
    for i, (sx, sy) in enumerate(source_points):
        phase = torch.exp(2j * math.pi * (sx * FX + sy * FY))
        A[:, i] = (pupil * phase).flatten() / math.sqrt(S)
    U, S_eig, _ = torch.svd_lowrank(A.cpu(), q=num_kernels + 20)
    kernels_freq = U[:, :num_kernels].to(device).reshape(pupil.shape[0], pupil.shape[1], num_kernels)
    eigenvalues = (S_eig[:num_kernels] ** 2).to(device)
    return kernels_freq, eigenvalues

def simulate_aerial_socs(mask, kernels_freq, eigenvalues):
    if mask.dim() == 2: mask = mask.unsqueeze(0)
    mask_fft = torch.fft.fft2(mask)
    aerial = torch.zeros_like(mask_fft, dtype=torch.float)
    for k in range(eigenvalues.shape[0]):
        field = torch.fft.ifft2(mask_fft * kernels_freq[:,:,k])
        aerial += eigenvalues[k] * (field.abs() ** 2)
    aerial = aerial.real
    uniform_fft = torch.fft.fft2(torch.ones_like(mask))
    clear = torch.zeros_like(aerial)
    for k in range(eigenvalues.shape[0]):
        field = torch.fft.ifft2(uniform_fft * kernels_freq[:,:,k])
        clear += eigenvalues[k] * (field.abs() ** 2)
    aerial /= (clear.real.mean() + 1e-10)
    return aerial.squeeze(0).clamp_(0)

In [ ]:
# Cell 3 – Precompute kernels (with compile)
print("Precomputing SOCS kernels (40 kernels)...")
padded_size = GRID_SIZE + 2 * int(GRID_SIZE * PAD_RATIO)
pupil, FX, FY = create_pupil_and_freq(padded_size)
source_pts = create_source_points()
kernels_freq, eigenvalues = precompute_socs_kernels(source_pts, pupil, FX, FY)

# Compile the hot path
simulate_aerial_socs = torch.compile(simulate_aerial_socs, mode="reduce-overhead")
print("SOCS ready + compiled.")

In [ ]:
# Cell 4 – Structure generators (same as before)
# (copy the 8 generate_ functions + structures dict from previous notebook)
# ... (omitted for brevity – paste the exact same code from my earlier message)

In [ ]:
# Cell 5 – Interactive resist tuning (now uses torch version)
# (same widget code as before, but it will now call the fast torch simple_resist)

In [ ]:
# Cell 6 – Raster + TV loss (unchanged)
def polygons_to_raster(polys, size=256):
    target = torch.zeros((size, size), device=device)
    for poly in polys:
        x1, y1, x2, y2 = poly
        x1, x2 = max(0, x1), min(size, x2)
        y1, y2 = max(0, y1), min(size, y2)
        if x2 > x1 and y2 > y1:
            target[y1:y2, x1:x2] = 1.0
    return target

def tv_loss(img, weight=5e-4):
    if img.dim() == 2:
        img = img.unsqueeze(0).unsqueeze(0)
    elif img.dim() == 3:
        img = img.unsqueeze(1)
    dx = torch.abs(img[:, :, :, 1:] - img[:, :, :, :-1])
    dy = torch.abs(img[:, :, 1:, :] - img[:, :, :-1, :])
    return weight * (dx.sum() + dy.sum()) / img.numel()

In [ ]:
# Cell 7 – Fast ILT optimization (M4-optimized)
def run_ilt_optimization(target_mask, n_steps=50, lr=0.1, tv_weight=5e-4):
    mask_ilt = target_mask.clone().unsqueeze(0).unsqueeze(0).requires_grad_(True)
    mask_ilt.data += 0.02 * torch.randn_like(mask_ilt.data)
    mask_ilt.data.clamp_(0.0, 1.0)
    
    optimizer = torch.optim.Adam([mask_ilt], lr=lr)
    loss_history = []
    
    for step in range(n_steps):
        optimizer.zero_grad()
        
        with torch.autocast(device_type="mps", dtype=torch.float16):
            padded, ph, pw, _ = pad_mask_for_pbc(mask_ilt.squeeze())
            aerial = unpad(simulate_aerial_socs(padded, kernels_freq, eigenvalues), ph, pw)
            resist = simple_resist(aerial, thresh=sl_thresh.value, steep=sl_steep.value,
                                   diff=sl_diff.value, load_sigma=sl_load_sig.value, gain=sl_gain.value)
            
            target = target_mask.unsqueeze(0).unsqueeze(0)
            bce = torch.nn.functional.binary_cross_entropy_with_logits(resist, target)
            tv = tv_loss(mask_ilt, weight=tv_weight)
            loss = bce + tv
        
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            mask_ilt.clamp_(0.0, 1.0)
        
        loss_history.append(loss.item())
        if step % 10 == 0 or step == n_steps-1:
            print(f"Step {step:3d} | BCE: {bce.item():.4f} | TV: {tv.item():.6f} | Total: {loss.item():.4f}")
    
    torch.mps.empty_cache()
    
    return {
        'mask_ilt': mask_ilt.squeeze().detach().cpu().numpy(),
        'aerial': aerial.detach().cpu().numpy(),
        'resist': resist.detach().cpu().numpy(),
        'loss_history': loss_history,
        'final_loss': loss_history[-1]
    }

In [ ]:
# Cell 8 – Batch generation (500 samples)
NUM_SAMPLES = 500
SAVE_DIR = os.path.join(ILT_DATA_DIR, datetime.now().strftime("%Y%m%d_%H%M%S"))
os.makedirs(SAVE_DIR, exist_ok=True)
dataset_list = []

print(f"Starting {NUM_SAMPLES} samples on 16 GB M4...")

for i in tqdm(range(NUM_SAMPLES)):
    name = np.random.choice(list(structures.keys()))
    target_mask, polygons = structures[name]()
    target_raster = polygons_to_raster(polygons)
    
    result = run_ilt_optimization(target_raster)
    
    sample = {
        'structure': name,
        'polygons': polygons,
        'target_raster': target_raster.cpu().numpy(),
        'ilt_mask': result['mask_ilt'],
        'aerial': result['aerial'],
        'resist': result['resist'],
        'loss_history': result['loss_history'],
        'final_loss': result['final_loss']
    }
    dataset_list.append(sample)
    
    if (i+1) % 50 == 0:
        np.savez(os.path.join(SAVE_DIR, f"sample_{i+1:04d}.npz"), **sample)

np.savez(os.path.join(SAVE_DIR, "ilt_dataset_500.npz"), samples=dataset_list)
print(f"Done → {SAVE_DIR}")

In [ ]:
# Cell 9 – Interactive Browser (design + ILT + overlay)
slider = widgets.IntSlider(value=0, min=0, max=len(dataset_list)-1, step=1, description='Sample:', continuous_update=False)
overlay = widgets.Checkbox(value=True, description='Overlay design contour')
out = widgets.Output()

def plot(idx):
    with out:
        clear_output(wait=True)
        s = dataset_list[idx]
        fig, ax = plt.subplots(1, 3, figsize=(18, 6))
        ax[0].imshow(s['target_raster'], cmap='gray'); ax[0].set_title('Design')
        ax[1].imshow(s['ilt_mask'], cmap='viridis', vmin=0, vmax=1); ax[1].set_title('ILT Mask')
        im = ax[2].imshow(s['resist'], cmap='viridis', vmin=0, vmax=2)
        if overlay.value:
            ax[2].contour(s['target_raster'] > 0.5, colors='red', linewidths=2)
        ax[2].set_title('Simulated Resist + Overlay')
        fig.colorbar(im, ax=ax[2], shrink=0.6)
        fig.suptitle(f"Sample {idx+1} | {s['structure']} | loss={s['final_loss']:.4f}")
        plt.show()

slider.observe(lambda c: plot(slider.value), names='value')
overlay.observe(lambda c: plot(slider.value), names='value')
display(widgets.VBox([widgets.HBox([slider, overlay]), out]))
plot(0)